In [36]:
import json
from pathlib import Path

import pandas as pd

In [37]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-sweep-new/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[1]

121 experiments


{'commit': 'c02cf9332600579abcdedd1202fa7d9ed3fa253e',
 'id': '85d3c1dde58cb4526fd3ca91730ccd6f8d8ad8e3',
 'name': 'crumb-vlei',
 'params': {'dataset': {'path': 'bdsaglam/musique-sweep',
   'name': 'answerable',
   'split': 'train'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.0,
   'system_prompt': 'ccot/format-thought.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'semi/ccot.json',
   'n_shot': 2,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.01,
  'f1': 0.019664098991685197,
  'fuzzy_match': 0.006666666666666667,
  '2hops': {'exact_match': 0.02,
   'f1': 0.034897047397047395,
   'fuzzy_match': 0.01},
  '3hops': {'exact_match': 0.01,
   'f1': 0.018333333333333333,
   'fuzzy_match': 0.01},
  '4hops': {'exact_match': 0.0,
   'f1': 0.005761916244674865,
   'fuzzy_match': 0.0},
  'gen_token_count': {'all': {'count': 300.0,
    'mean': 36.35,
    'std': 354.2041446496025,
    'min': 1.0,
    '25%': 7.0,
    '50%': 12.0,
    '75%': 20.0,
    'ma

In [38]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-sweep")
    # & (df["metrics.f1"] > 0.1)
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

93 experiments after preprocessing


In [39]:
def parse_qa_technique(row):
    base_technique = row["params.qa.system_prompt"].split("/")[0].upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [40]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.n_sc",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.n_sc,params.qa.temperature,params.run
0,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,1,0.0,"(1,)"
1,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,1,0.5,"(1,)"
2,ccot/format-thought.txt,cq.txt,semi/ccot.json,0,1,1.0,"(1,)"
3,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,1,0.0,"(1,)"
4,ccot/format-thought.txt,cq.txt,semi/ccot.json,1,1,0.1,"(1,)"
...,...,...,...,...,...,...,...
88,direct/format-few.txt,cq.txt,semi/direct.json,2,1,1.0,"(1,)"
89,direct/format-few.txt,cq.txt,semi/direct.json,3,1,0.0,"(1,)"
90,direct/format-few.txt,cq.txt,semi/direct.json,3,1,0.5,"(1,)"
91,direct/format-few.txt,cq.txt,semi/direct.json,3,1,0.7,"(1,)"


In [41]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-thought.txt
	cok/adapted.txt
	cot/format-thought.txt
	cte/format-triples-ere-few.txt
	cte/format-triples-few.txt
	direct/format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	semi/ccot.json
	semi/cok.json
	semi/cot.json
	semi/cte-triples.json
	semi/direct.json
- params.qa.n_shot
	0
	1
	2
	3
- params.qa.n_sc
	1
- params.qa.temperature
	0.0
	0.5
	1.0
	0.1
	0.7
- params.run
	(1,)


In [42]:
df.to_json('exps.jsonl', orient='records', lines=True)